# Generating Populations and Species
Assuming that the civilization is on the verge of moving from a phaze 0 to phase 1 civ.
* baseline species
* n populations (pops)
* population varience (within the norms of the species)
* allegiance groups to factions

Starting with an example user form data

In [1]:

from numpy import random, round, interp, linspace
import pickle
from sklearn.cluster import KMeans
import altair as alt
import sys, os
from sklearn.cluster import KMeans

import pandas as pd


origional `data` comes from the user via the form in `forms.py`

In [2]:
starting_attributes = ['conformity', 'literacy', 'aggression', 'constitution']

data = {
    'conformity':0.4,
    'literacy':0.7,
    'aggression':0.5,
    'constitution':0.4,
    'starting_pop': 100
}

def build_species(data):
    species = {}
    for attr in starting_attributes:
        species[attr] = data[attr]
    return species
    
species = build_species(data)
species


{'conformity': 0.4, 'literacy': 0.7, 'aggression': 0.5, 'constitution': 0.4}

In [3]:
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f

## Creating groups of populations, all with different attributes

Populations vary based on conformity, some populations are more literate, some are more aggressive. 

In [4]:
pop_std = .2 * (1-species['conformity'])
print("the population conformity: ", pop_std)

def vary_pops(species):
    pop = {}
    for k in list(species.keys()):
        pop[k] = abs(round(random.normal(species[k], pop_std),3))
    return pop

pops = pd.DataFrame([vary_pops(species) for i in range(data['starting_pop'])])
pops

the population conformity:  0.12


,conformity,literacy,aggression,constitution
0,0.457,0.740,0.580,0.228
1,0.268,0.651,0.516,0.473
2,0.181,0.689,0.637,0.478
3,0.200,0.826,0.435,0.387
4,0.561,0.642,0.443,0.475
...,...,...,...,...
95,0.416,0.666,0.419,0.417
96,0.456,0.702,0.462,0.074
97,0.369,0.814,0.717,0.433
98,0.553,0.773,0.446,0.483


The end result is that population factions are grouped by the idealogical differences of the people in them. This gives each faction a distinct culture. 

## Dividing pops into factions

I feel like there isn't a way to pick an ideal number of factions procedurally,  so until I think of something better I'm just going to go with an arbitrary range based on `population_conformity`.

The amount of different nations is relative to `1-population_conformity` and then scaled out over a number of steps defined as `n_steps`

In [5]:
n_steps = 6

def get_n_factions(n_steps):
    x = interp(
        (1-data["conformity"]),
            linspace(0, 1, num=n_steps),
            [i for i in range(n_steps)]
        )
    return int(round(x))

get_n_factions(n_steps)

3

In [6]:
kmeans = KMeans(n_clusters=n_steps).fit(pops[starting_attributes])
pops['faction_no'] = kmeans.labels_

In [7]:
factions = pd.DataFrame(kmeans.cluster_centers_, columns=starting_attributes)
factions['name'] = factions['conformity'].apply(lambda x: f.make_word(1))
factions

,conformity,literacy,aggression,constitution,name
0,0.438789,0.707474,0.597684,0.275263,Khawr
1,0.447200,0.842667,0.399733,0.380467,Khlo
2,0.386500,0.496333,0.628333,0.553000,Kmer
3,0.335421,0.593526,0.420947,0.362632,Nont
4,0.222500,0.737900,0.530250,0.460600,Ndo
5,0.547286,0.674048,0.481333,0.445667,Twer


So individual `pops` vary slightly, relative to the `conformity`. And `conformity` also determines the number of factions in the group. 

In [8]:
chart = alt.Chart(pops).mark_circle().encode(x='literacy',y='aggression',color='faction_no:N')
chart

alt.Chart(...)

The cluster centers can represent the 'zeitgeist' of that faction's culture. 

# Other population attributes

## Other attributes that get added
These items aren't used to calculate factions as they refer to resources, not values. These are calculated at the creation of a `pop` and later can be updated. 

* `industry` = `aggression` + `constitution`
* `wealth` = sum(`literacy` + `industry`)/2



In [10]:
pops['industry'] = (pops['aggression'] + pops['constitution'])/2
pops['wealth'] = (pops['literacy'] + pops['industry'])/2
pops['target_score'] = (1-pops['conformity']) - pops['aggression']
pops

,conformity,literacy,aggression,constitution,faction_no,industry,wealth,target_score
0,0.457,0.740,0.580,0.228,0,0.4040,0.57200,-0.037
1,0.268,0.651,0.516,0.473,4,0.4945,0.57275,0.216
2,0.181,0.689,0.637,0.478,4,0.5575,0.62325,0.182
3,0.200,0.826,0.435,0.387,4,0.4110,0.61850,0.365
4,0.561,0.642,0.443,0.475,5,0.4590,0.55050,-0.004
...,...,...,...,...,...,...,...,...
95,0.416,0.666,0.419,0.417,3,0.4180,0.54200,0.165
96,0.456,0.702,0.462,0.074,0,0.2680,0.48500,0.082
97,0.369,0.814,0.717,0.433,0,0.5750,0.69450,-0.086
98,0.553,0.773,0.446,0.483,5,0.4645,0.61875,0.001


In [11]:
chart = alt.Chart(pops).mark_circle().encode(x='wealth',y='industry',color='faction_no:N')
chart

alt.Chart(...)

In [12]:
chart = alt.Chart(
        pops.reset_index(drop=False)
        ).mark_bar(size=4).encode(
            x=alt.X('index:N', sort='-y'),
            y='wealth',
            color='faction_no:N').properties(width=600,title='Some populations make better targets than others')
chart

alt.Chart(...)

In [13]:
chart.encode(
            x=alt.X('faction_no:N', sort='-y'),
            y='wealth',
        
            color='faction_no:N').properties(title='Some factions have greater wealth')

alt.Chart(...)

In [14]:
# chart_pivot = pops.unstack().reset_index(drop=False).rename(columns={'level_0':'attribute', 'level_1':'item', 0:'value'})
# chart_pivot

# Faction Loyalty
* More aggressive pops will have less loyalty. 
* populations with values more distant to the faction will have less loyalty. 

How do you calculate the difference between values of two groups?

In [15]:
def compare_values(values,gr1,gr2):
    distance = []
    for v in values:
        distance.append(abs(gr1.get(v,0)-gr2.get(v,0)))
    return sum(distance)/len(distance)

compare_values(starting_attributes,
    pops.loc[0].to_dict(),
    pops.loc[2].to_dict())

0.15850000000000003

The average of the distance between values indicates the idealogical separation between two peoples. The same can be used to measure a people and it's faction, as well as the animosity between two factions. 

compare values determines how similar groups are, then `1-compare` is how distant they are. This is used to place the starting loyalty. 

In [18]:
def get_faction_loyalty(x):
    g1 = pops.loc[x].to_dict()
    f2 = factions.loc[int(g1['faction_no'])].to_dict()
    return compare_values(starting_attributes,g1,f2)


pops['faction_loyalty'] = [(1-get_faction_loyalty(i)) for i in pops.index]

alt.Chart(pops).mark_point().encode(
            x=alt.X('aggression:Q', sort='-y'),
            y=alt.Y('faction_loyalty:Q',scale=alt.Scale(domain=(0.8, 1))),
            color='faction_no:N').properties(title='Depending on the conformity, and aggression of people, the loyalty could be high or low.')


alt.Chart(...)

Beware of populations that have high aggression and low faction loyalty

This applies to factions as well. 

Note that this doesn't need to be stored, it can be calculated on the fly so long as the `values` for each party are available. 

In [70]:
[[(f1['name'],f2['name'],1-compare_values(starting_attributes,f1,f2))
    for f1 in factions.to_dict("records")] 
    for f2 in factions.to_dict("records")]

[[('Mia', 'Mia', 1.0),
  ('Greens', 'Mia', 0.8805052681992337),
  ('Chaic', 'Mia', 0.8796697198275862),
  ('Weyn', 'Mia', 0.8846571618037136),
  ('Trubc', 'Mia', 0.883621473354232),
  ('Pi', 'Mia', 0.8885145888594165)],
 [('Mia', 'Greens', 0.8805052681992337),
  ('Greens', 'Greens', 1.0),
  ('Chaic', 'Greens', 0.9074149305555556),
  ('Weyn', 'Greens', 0.8676463675213675),
  ('Trubc', 'Greens', 0.8987411616161617),
  ('Pi', 'Greens', 0.8686485042735043)],
 [('Mia', 'Chaic', 0.8796697198275862),
  ('Greens', 'Chaic', 0.9074149305555556),
  ('Chaic', 'Chaic', 1.0),
  ('Weyn', 'Chaic', 0.8343016826923078),
  ('Trubc', 'Chaic', 0.8913053977272727),
  ('Pi', 'Chaic', 0.9255420673076923)],
 [('Mia', 'Weyn', 0.8846571618037136),
  ('Greens', 'Weyn', 0.8676463675213675),
  ('Chaic', 'Weyn', 0.8343016826923078),
  ('Weyn', 'Weyn', 1.0),
  ('Trubc', 'Weyn', 0.8422587412587412),
  ('Pi', 'Weyn', 0.8884423076923077)],
 [('Mia', 'Trubc', 0.883621473354232),
  ('Greens', 'Trubc', 0.8987411616161617),